In [1]:
from transformers import pipeline, set_seed
generator = pipeline("text-generation", model="gpt2")
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, but what I'm really doing is making a human-readable document. There are other languages, but those are"},
 {'generated_text': "Hello, I'm a language model, not a syntax model. That's why I like it. I've done a lot of programming projects.\n"},
 {'generated_text': "Hello, I'm a language model, and I'll do it in no time!\n\nOne of the things we learned from talking to my friend"},
 {'generated_text': "Hello, I'm a language model, not a command line tool.\n\nIf my code is simple enough:\n\nif (use (string"},
 {'generated_text': "Hello, I'm a language model, I've been using Language in all my work. Just a small example, let's see a simplified example."}]

In [2]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors="pt")
output = model(**encoded_input)


In [10]:
import datasets
train_10 = datasets.load_dataset("roneneldan/TinyStories", split="train[:10%]")
len(train_10)


Repo card metadata block was not found. Setting CardData to empty.


211972

In [5]:
def tokenization(example):
    return tokenizer(
        example["text"],
        max_length=model.config.max_position_embeddings,
        truncation=True,
        return_attention_mask=False,
    )


# Batched tokenization. Can feed into model for outputs/logits to be softmaxed.
ds = train_10.map(
    tokenization,
    batched=True,
    num_proc=8,
)


Map (num_proc=8):   0%|          | 0/211972 [00:00<?, ? examples/s]

In [6]:
ds


Dataset({
    features: ['text', 'input_ids'],
    num_rows: 211972
})

In [25]:
ds[0]["text"], ds[0]["input_ids"]


('One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.',
 [3198,
  1110,
  11,
  257,
  1310,
  2576,
  3706,
  20037,
  1043,
  257,
  17598,
  287,
  607,
  2119,
  13,
  1375,
  2993,
  340,
  373,
  2408,
  284,
  711,
  351,
  340,
  780,
  340,
  373,
  7786,
  13,
  20037,
  2227,
  284,
  2648,
  262,
  17598,
  351,
  607,
  1995,
  11

In [9]:
len(ds[0]["input_ids"]), len(ds[0]["text"].split(" "))


(162, 132)

In [11]:
encoded_input


{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [33]:
tokenizer("Replace"), tokenizer("me"), tokenizer("by"), tokenizer("Replace me by"), tokenizer


Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


({'input_ids': [3041, 5372], 'attention_mask': [1, 1]},
 {'input_ids': [1326], 'attention_mask': [1]},
 {'input_ids': [1525], 'attention_mask': [1]},
 {'input_ids': [3041, 5372, 502, 416], 'attention_mask': [1, 1, 1, 1]},
 GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
 })

In [24]:
tokenizer.convert_ids_to_tokens([3041, 5372, 502, 416, 597, 2420]), tokenizer.convert_ids_to_tokens(encoded_input["input_ids"][0])


(['Re', 'place', 'Ġme', 'Ġby', 'Ġany', 'Ġtext'],
 ['Re', 'place', 'Ġme', 'Ġby', 'Ġany', 'Ġtext', 'Ġyou', "'d", 'Ġlike', '.'])

In [26]:
generator("One day, a little girl named Lily found a needle in her room.", max_length=30, num_return_sequences=5)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'One day, a little girl named Lily found a needle in her room. She jumped out, grabbed a few items, pulled up the cover, got'},
 {'generated_text': 'One day, a little girl named Lily found a needle in her room. It was an old tube in her back, with her eyes closed, her'},
 {'generated_text': 'One day, a little girl named Lily found a needle in her room. She immediately had a good dream — a very different one.\n\nHer'},
 {'generated_text': 'One day, a little girl named Lily found a needle in her room. She was taken to the hospital and treated with antibiotics.\n\nShe has'},
 {'generated_text': 'One day, a little girl named Lily found a needle in her room.\n\n"What," she said, holding out her hand to stop the'}]

In [82]:
import numpy as np
import scipy
tmp = output.last_hidden_state[0][0].detach().numpy()
type(output), output.last_hidden_state.shape, tmp.max(), tmp.min(), ((tmp > -1) & (tmp < 1)).sum(), scipy.special.softmax(tmp).sum(), scipy.special.softmax(tmp).argmax()


(transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions,
 torch.Size([1, 10, 768]),
 46.90026,
 -12.414162,
 755,
 1.0,
 496)

In [76]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2", return_dict_in_generate=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
input_ids = tokenizer("This is a scary story.", return_tensors="pt").input_ids
# Adjust beams.
outputs = model.generate(input_ids, num_beams=1, num_return_sequences=1, output_scores=True, max_length=50)  # , length_penalty=0)
# outputs


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [77]:
tokenizer.convert_ids_to_tokens(outputs["sequences"][0]), len(outputs["sequences"][0])


(['This',
  'Ġis',
  'Ġa',
  'Ġscary',
  'Ġstory',
  '.',
  'ĠI',
  "'m",
  'Ġnot',
  'Ġsure',
  'Ġwhat',
  'Ġto',
  'Ġmake',
  'Ġof',
  'Ġit',
  '.',
  'ĠI',
  "'m",
  'Ġnot',
  'Ġsure',
  'Ġwhat',
  'Ġto',
  'Ġdo',
  '.',
  'ĠI',
  "'m",
  'Ġnot',
  'Ġsure',
  'Ġwhat',
  'Ġto',
  'Ġdo',
  '.',
  'ĠI',
  "'m",
  'Ġnot',
  'Ġsure',
  'Ġwhat',
  'Ġto',
  'Ġdo',
  '.',
  'ĠI',
  "'m",
  'Ġnot',
  'Ġsure',
  'Ġwhat',
  'Ġto',
  'Ġdo',
  '.',
  'ĠI',
  "'m"],
 50)

In [79]:
outputs["scores"], len(outputs["scores"]), outputs["scores"][0].shape, outputs["scores"][0][0].argmax(), tokenizer.convert_ids_to_tokens([outputs["scores"][0][0].argmax().item()]), tokenizer.convert_ids_to_tokens([outputs["scores"][1][0].argmax().item()]), [tokenizer.convert_ids_to_tokens([outputs["scores"][i][0].argmax().item()]) for i in range(44)]


((tensor([[-145.6897, -144.1550, -145.3307,  ..., -156.3612, -160.2338,
           -138.2815]]),
  tensor([[-163.4249, -162.4569, -167.6689,  ..., -171.3964, -164.3011,
           -164.5081]]),
  tensor([[-144.9066, -142.9343, -147.6708,  ..., -150.0803, -149.4902,
           -145.3383]]),
  tensor([[-140.2967, -139.6254, -149.2209,  ..., -146.1835, -147.8075,
           -141.7704]]),
  tensor([[-132.5165, -132.1021, -141.0429,  ..., -139.8127, -142.6759,
           -134.9256]]),
  tensor([[-111.0757, -109.2763, -116.0065,  ..., -114.7578, -117.3087,
           -111.3735]]),
  tensor([[-140.9013, -139.3839, -145.6808,  ..., -148.9513, -152.1206,
           -142.7282]]),
  tensor([[  1.7674,   3.1039,  -6.8053,  ...,  -7.1692, -10.1013,  -2.4498]]),
  tensor([[ -93.5340,  -91.8021,  -98.7108,  ...,  -97.7029, -102.4045,
            -94.2247]]),
  tensor([[-63.0810, -62.7109, -71.2882,  ..., -76.9252, -77.8038, -66.9183]]),
  tensor([[-152.7949, -150.4533, -152.4851,  ..., -164.2371, -16

In [81]:
outputs.last_hidden_state


AttributeError: 'GreedySearchDecoderOnlyOutput' object has no attribute 'last_hidden_state'